<a href="https://colab.research.google.com/github/silverstar0727/DSC-ML-Research-Study/blob/main/%ED%95%A9%EC%84%B1%EA%B3%B1_%EC%98%A4%ED%86%A0%EC%9D%B8%EC%BD%94%EB%8D%94_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import *

# Hyperparameters

In [ ]:
""" configuration json을 읽어들이는 class """
class Config(dict): 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

    @classmethod
    def load(cls, file):
        with open(file, 'r') as f:
            config = json.loads(f.read())
            return Config(config)

In [ ]:
config = Config({
    'batch_size': 256,
    'learning_rate': 0.0002,
    'num_epoch': 5
})

# Model
- Encoder

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, config): # config 받아오기
    suepr(Encoder, self).__init__()
    self.config = config

    ## layer1 지정
    self.layer1 = tf.keras.Sequential()
    self.layer1.add(tf.keras.layers.Conv2D(filters=16, kernel_size=3, strides=2, padding='same', activation='relu'))
    self.layer1.add(tf.keras.layers.BatchNormalization())
    self.layer1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='relu'))
    self.layer1.add(tf.keras.layers.BatchNormalization()
    self.layer1.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu'))
    self.layer1.add(tf.keras.layers.BatchNormalization()
    self.layer1.add(tf.keras.layers.MaxPool2D((2,2)))

    ## layer2 지정
    self.layer2 = tf.keras.Sequential()
    self.layer2.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2, padding='same', activation='relu'))
    self.layer2.add(tf.keras.layers.BatchNormalization())
    self.layer2.add(tf.keras.layers.MaxPool2D((2,2)))
    self.layer2.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=2, padding='same', activation='relu'))

  def call(self, x):
    # layer1과 layer2의 조합
    out = self.layer1(x)
    out = self.layer2(x)
    out = tf.reshape(out, (self.config.batch_size, -1))

    return out

- Decoder

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, config):
    super(Decoder, self).__init__()
    self.config = config

    ## layer1 지정
    self.layer1 = tf.keras.Sequential()
    self.layer1.add(tf.keras.layers.Convolution2DTranspose(filters=128, kernel_size=3, strides =2 ,padding =, activation = 'relu'))
    self.layer1.add(tf.keras.layers.BatchNormalization())
    self.layer1.add(tf.keras.layers.Convolution2DTranspose(filters=64, kernel_size=3, strides = 2,padding =, activation = 'relu'))
    self.layer1.add(tf.keras.layers.BatchNormalization())

    ## layer2 지정
    self.layer2 = tf.keras.Sequential()
    self.layer2.add(tf.keras.layers.Convolution2DTranspose(filters=16, kernel_size=3, strides = 2,padding ='same', activation = 'relu'))
    self.layer2.add(tf.keras.layers.BatchNormalization())
    self.layer2.add(tf.keras.layers.Convolution2DTranspose(filters=1, kernel_size=3, strides =2 ,padding ='same', activation = 'relu'))

  def call(self, x):
    # layer1과 layer2의 조합
    out = tf.reshape(x, (self.config.batch_size, 256, 7, 7))
    out = self.layer1(out)
    out = self.layer2(out)

    return out

- 완성

In [ ]:
class Model(tf.keras.Model):
  def __init__(self, config):
    super(Model, self).__init__()
    self.config = config

    self.Encoder = Encoder(self.config)
    self.Decoder = Decoder(self.config)

  def call(self, input):
    out = Encoder(input)
    out = Decoder(out)

    return out

## compile

In [ ]:
model = Model()
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = config.learning_rate),
              loss = tf.keras.losses.MSE(),
              metrics = [tf.keras.metrics.mse()])